In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely import LineString, Point, Polygon
from tobler.util import h3fy
import math
import numpy
import pulp
import utm
import statisticoperations as sop
import objects_location

In [ ]:
# Define your parameters

WIND_DIRECTION = 'south_west' # set an area destination

# set percent od every area you need
landuse_percent = {
    'living': 0.543,
    'social': 0.066,
    'industrial': 0.061,
    'transport': 0.077,
    'recreation': 0.253
}

In [ ]:
# load files of the territory and city
territory = gpd.read_file('original_data/siver.geojson').to_crs('EPSG:4326')

city = gpd.read_file('original_data/СНП Сиверского городского поселения.geojson').to_crs('EPSG:4326')
region = gpd.read_file('original_data/Сиверское городское поселение.geojson').to_crs('EPSG:4326')

# all buildings - living
buildings = gpd.read_file('collected_data/balanced_buildings.geojson')
buildings['building:levels'] = buildings['building:levels'].astype(int)
buildings_p = buildings.copy()
buildings_p['geometry'] = buildings_p['geometry'].centroid.to_crs('EPSG:4326')

In [ ]:
# CONSTANTS FOR THIS PROJECT
utm_system = utm.from_latlon(territory.loc[0, 'geometry'].centroid.y, territory.loc[0, 'geometry'].centroid.x) # find UTM zone
LOCAL_CRS = 'EPSG:326' + str(utm_system[2])
centre_territory = territory.to_crs(LOCAL_CRS).loc[0, 'geometry'].centroid
territory_area_he = territory.to_crs(LOCAL_CRS).area / 10000

In [ ]:
# create hexagones for future analysis hex_area = 1.26 he
hex_city = h3fy(city, resolution=10, clip=True, buffer=False, return_geoms=True).reset_index()
hex_territory = h3fy(territory, resolution=10, clip=True, buffer=False, return_geoms=True).to_crs(LOCAL_CRS).reset_index()

# set_default_parameters
hex_territory['landuse'] = 'not_defined'
hex_territory['func_zone'] = 'not_defined'

hex_territory['score_industrial'] = 0
hex_territory['score_social'] = 0
hex_territory['score_middle_living'] = 0
hex_territory['score_low_living'] = 0
hex_territory['score_cottage_living'] = 0
hex_territory['score_garden'] = 0
hex_territory['score_park'] = 0
hex_territory['score_green_buffer'] = 0
hex_territory['fixed'] = 0

# get base data merge hex and floors
for i in range(len(hex_city)):
        # Выбор точек внутри полигона
    points_within_polygon = buildings_p[buildings_p.geometry.within(hex_city.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('is_living')['building:levels'].mean().reset_index()
    hex_city.loc[i, 'Floors'] = grouped_values['building:levels'].max()
    grouped_values['population'] = 0